In [2]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [1]:
nav = "navigation-pagination__list" #ul class

In [3]:
url = "https://www.bgateway.com/events/p1?"

In [4]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [5]:
driver.get(url)

In [6]:
soup = bs(driver.page_source,'lxml')

In [7]:
last_page = int(soup.find('ul',{'class':nav}).find_all('li')[-1].text.strip().split('\n')[-1].strip())
last_page

15

In [8]:
buga = []
for i in range(1,last_page+1):
    url = f"https://www.bgateway.com/events/p{i}?"
    driver.get(url)
    time.sleep(10)
    #section = "row justify-content-center"
    search_result = soup.find('div',{'class':'search-results py-4'})
    each = search_result.find_all('div',{'class':"search-results__item py-4 border-bottom"})
    
    for l in each:
        title = l.find('h3').text.strip()
        link = l.find('a')['href'].strip()
        metadata =l.find_all('div',{'class':'search-results__metadata'})
        excerpt = l.find('div',{'class':'search-results__excerpt'}).text.strip()
        if excerpt=='' or str(excerpt)=='None':
            event_info=''
        else:
            event_info=excerpt
        try:
            dat,daytime,nationality,ven = [s.text.strip() for s in metadata]
        except:
            dat,daytime,ven = [s.text.strip() for s in metadata]
            nationality = 'NA'
        #***********
        date__=dat.replace('–','-')
        if date__=='':
            start_date=end_date=''
        else:
            mad=date__
            try:
    #             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
    #             mad=ma.group()

                if '-' in mad:
                    st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                    en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

                elif 'and'in mad:
                    st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                    en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

                elif '&' in mad:
                    st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                    en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

                else:
                    st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                    en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                #print(date__,st,en)
                ####################
                if any(c.isalpha() for c in st)==False:
                #      print('leg')
                    pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
                    sapa=pa.group()
                    start=st+' '+sapa
                    end=en
                #                        02        April      2022
                elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
                 #   print('awujale')
                    start=st
                    end=en

                #*                        April       02      2022
                elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
                #       print('ala')
                    pa=re.search('(\d{4})',st).group()
                    sa=re.search('([A-Sa-z]{3,9})',st).group()
                    ta=re.search('(\d{1,2})',st).group()
                    start=ta+' '+sa+' '+pa
                    #
                    ba=re.search('(\d{4})',en).group()
                    ca=re.search('([A-Sa-z]{3,9})',en).group()
                    da=re.search('(\d{1,2})',en).group()
                    end=da+' '+ca+' '+ba     

                #                      02     2022
                elif re.search('\d{1,2}\s+\d{4}',en):
                #      print('kum')
                    pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                    sa=re.search('([A-Sa-z]{3,9})',st).group()
                    ta=re.search('(\d{1,2})',st).group()

                    start=ta+' '+sa+' '+pa
                    #
                    ba=re.search('(\d{4})',en).group()
                    ca=re.search('([A-Sa-z]{3,9})',st).group()
                    da=re.search('(\d{1,2})',en).group()
                    end=da+' '+ca+' '+ba

                #                      02         April
                elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
                #        print('is')
                    pa=re.search('(\d{4})',en)#, maxsplit=0)
                    sapa=pa.group()
                    start=st+' '+sapa
                    end=en

                #*                      April           02
                elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
                #      print('bad')
                    pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                    sa=re.search('([A-Sa-z]{3,9})',st).group()
                    ta=re.search('(\d{1,2})',st).group()

                    start=ta+' '+sa+' '+pa
                    #
                    ba=re.search('(\d{4})',en).group()
                    ca=re.search('([A-Sa-z]{3,9})',en).group()
                    da=re.search('(\d{1,2})',en).group()
                    end=da+' '+ca+' '+ba

                else:
                #     print('shik')
                    start=end=''
                if start=='':
                    start_date=end_date=''
                else:
                    pick=[start, end]
                    spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
                    date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
                    start_date=date_[0]
                    end_date=date_[1]
            except:
                start_date=end_date=''
    #**************daytime******
        tim = daytime.replace('–','-')
        if '-' in tim:
            st=tim.split('-')[0].strip()
            et=tim.split('-')[1].strip()
            
        elif 'to' in tim:
            st=tim.split('to')[0].strip()
            et=tim.split('to')[1].strip()
        else:
            st=tim.strip()
            et=''
        limit=[st,et]
        if limit[1]!='':
            sweet=[datetime.strptime(v, '%H:%M') for v in limit]
            swets=[z.strftime('%I:%M%p') for z in sweet]
        else:
            try:
                sweet=[datetime.strptime(limit[0], '%H:%M'), '']
                swets=[sweet[0].strftime('%I:%M%p'),'']
            except:
                swets=''
        if swets=='':
            time_st=''
        else:
            time_st={'type': 'general',
                 'start_time':swets[0],
                 'end_time': swets[1],
                 'timezone': '',
                 'days': 'all'}
         ###############COUNTYR,GPU#################
        def country(locale):
            try:
                google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                google_map_url=driver.find_element_by_id('lu_map').click()
                time.sleep(1)
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
                return(country)
            except Exception as e:
                print(e)
                return("")

        ####################################
        def get_google_map_url(location):
            try:
                google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                google_map_url=driver.find_element_by_id('lu_map').click()
                time.sleep(1)
                google_map_url=driver.current_url
                return(google_map_url)
            except Exception as e:
                print(e)
                return("")
    
        if 'Web' in nationality or 'Online' in nationality:
            on_off='1'
            city=''
            country=''
            venue=''
            googlePlaceUrl=''
        else:
            on_off='0'
            city=ven
            country=country(city)
            venue=ven
            googlePlaceUrl=get_google_map_url(venue)

        #print(nationality,venue,city,on_off)
        org_pro='Business Gateway is a publicly funded service contributing to the economic well being of Scotland by providing access to free business support services.'
        org_name='Business Gateway'
        org_web='https://www.bgateway.com/'
        ticket_list=''
        logo=''
        sponsors=''
        agendalist=''
        type_=''
        category=''
        event_web=link
        mail_=''
        if time_st=='':
            time_=''
        else:
            time_=json.dumps(time_st)
        Speakerlist=''
        buga.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsors,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])
    time.sleep(10)
print('done')

done


In [9]:
buga

[['https://www.bgateway.com/events/improve-your-search-engine-ranking-webinar-23460',
  'Improve Your Search Engine Ranking - Webinar',
  '2022-04-18',
  '2022-04-18',
  '{"type": "general", "start_time": "02:00PM", "end_time": "04:30PM", "timezone": "", "days": "all"}',
  'This webinar aims to improve your understanding of how search engines prioritise results.',
  '',
  'Business Gateway is a publicly funded service contributing to the economic well being of Scotland by providing access to free business support services.',
  'Business Gateway',
  'https://www.bgateway.com/',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'https://www.bgateway.com/events/improve-your-search-engine-ranking-webinar-23460',
  '',
  '',
  '',
  '1'],
 ['https://www.bgateway.com/events/marketing-1-2-1-sessions-23560',
  'Marketing 1-2-1 Sessions',
  '2022-04-19',
  '2022-04-19',
  '{"type": "general", "start_time": "09:00AM", "end_time": "05:00PM", "timezone": "", "days": "all"}',
  'Book an appointment

In [10]:
buga_dict=buga


buga_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=buga_dict)
buga_df.to_csv("buga.tsv", sep = '\t',index=False)
